In [4]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Read the Excel file
df = pd.read_excel("staff.xlsx")

# Initialize the Dash app
app = dash.Dash(__name__)

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    dcc.Dropdown(
        id='project-dropdown',
        options=project_options,
        value=project_options[0]['value']  # Default value
    ),
    dcc.Graph(id='deployment-plan')
])

# Define callback to update the deployment plan based on selected project
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value')]
)
def update_plan(selected_project):
    # Filter the DataFrame based on the selected project
    filtered_data = df[df['Project'] == selected_project]

    # Get unique job titles for the selected project
    unique_jobs = filtered_data['Job'].unique()

    # Create a bar chart for each job title
    data = []
    for job in unique_jobs:
        job_data = filtered_data[filtered_data['Job'] == job]
        months_worked = list(job_data['Month'])
        
        # Create a horizontal bar chart for each job title
        bar = go.Bar(
            y=[job],
            x=months_worked,
            orientation='h',
            name=job,
            marker=dict(
                color='rgba(50, 171, 96, 0.6)',
                line=dict(
                    color='rgba(50, 171, 96, 1.0)',
                    width=1)
            )
        )
        data.append(bar)

    # Define layout for the horizontal bar chart
    layout = go.Layout(
        barmode='stack',
        xaxis=dict(title='Months'),
        yaxis=dict(title='Job Titles'),
        title=f"Deployment Plan for {selected_project}"
    )

    return {'data': data, 'layout': layout}

if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050, debug=True)
